### 이번 실습에서는 다음 내용들을 배웁니다.


- 이제까지 배웠던 내용들을 합쳐서 원하는 데이터를 가져오는 코드를 작성합니다.


- 원하는 지수 데이터를 불러오는 코드를 작성해봅니다.

### 날짜 추출

- td 태그중에 원하는 정보만을 따로 가져와야 합니다. (표를 자세히 보시면 왜 54개인지 알 수 있습니다.)


- 날짜 태그의 규칙을 찾아서 td 태그들 중에 날짜를 가져옵니다.


- 그런데 해당 페이지위에서 날짜 정보가 있는 tag에 대한 규칙을 찾기가 어렵습니다. 이럴 때 XPath 정보를 이용하여 바로 검색할 태그를 지정할 수 있습니다.


- 아래 그림과 같이 우클릭을 해서 Copy XPath를 눌러줍니다.

![sise_xpath](webcrawling/get_xpath.png)

In [19]:
# date에 대한 xpath
#/html/body/div/table[1]/tbody/tr[3]/td[1] 
import requests
import bs4

page_no = 2
page_url = f"https://finance.naver.com/sise/sise_index_day.naver?code=KPI200&page={page_no}"

source = requests.get(page_url).text
source = bs4.BeautifulSoup(source)
source

dates = source.find_all("td", class_="date")
dates

[<td class="date">2023.02.14</td>,
 <td class="date">2023.02.13</td>,
 <td class="date">2023.02.10</td>,
 <td class="date">2023.02.09</td>,
 <td class="date">2023.02.08</td>,
 <td class="date">2023.02.07</td>]

In [20]:
dates[0].text

'2023.02.14'

In [21]:
# tag를 통해 가져온 다음에 텍스트를 가져옵니다. 데이터는 text 형태로 tag 사이에 있습니다.
# tag 사이에 껴있는 텍스트를 가져올 때 .text를 사용합니다.
date_list = []

for date in dates:
    date_list.append(date.text)
    
date_list

['2023.02.14',
 '2023.02.13',
 '2023.02.10',
 '2023.02.09',
 '2023.02.08',
 '2023.02.07']

- 날짜를 크롤링하는데 성공했습니다!!


- 지금은 문자열인 이 정보를 실제 datetime 정보로 변환하여 사용하고 싶습니다. (이후에 pandas에서 결합하여 사용할 예정)


- YYYY.MM.DD 형식의 문자열을 datetime 데이터 타입으로 변환합니다.

In [ ]:
import datetime as dt

In [ ]:
# 가져온 datetime을 YYYY.MM.DD 형태로 변환합니다.
yyyy = 
mm = 
dd = 

this_date= dt.date(yyyy, mm, dd)
this_date

#### 날짜정보를  date 타입으로 변경하는 함수 정의

In [ ]:
# 날짜 변경 하는 기능은 계속해서 사용할 것이기 때문에, 함수로 정의합니다. 나중에 어떻게 코드상에서 사용되는지 눈여겨 보시면 좋습니다.

def date_format(d):

    
    
    
    return this_date

### 체결가(종가) 추출

#### 페이지 상의 날짜와 종가정보 전체 추출

In [22]:
prices = source.find_all("td", class_="number_1")

price_list = []
for price in prices[::4]:
    price_list.append(price.text)
price_list

['323.47', '321.75', '323.69', '324.90', '325.63', '320.55']

In [23]:
len(date_list)

6

In [24]:
len(price_list)

6

- 페이지에서 표의 태그들을 살펴보면, 체결가 이후로 전일비/등락률/거래량/거래대금 들도 같은 태그를 공유하고 있어, 4개씩 증가하는 것을 알 수 있습니다.


- 이것을 규칙으로 number_1으로 추출한 태그는 24개이지만, 이 중에서 4의 배수로 건너뛰면서 추출하면 바로 체결가를 가져올 수 있다는 것을 확인할 수 있습니다.

In [ ]:
# dates 데이터를 활용하여 number_1에서 종가를 추출해봅니다.
for n in range(len(dates)):

    
    
    
    
    print(this_date, this_close)

- 이제 크롤러의 마지막 관문이 남았습니다


- 현재까지 하나의 페이지에서 크롤링 하는 것을 완료하였습니다.


- 이제 이 작업을 원하는 페이지까지(또는 끝까지) **반복** 진행하면 됩니다.


- 그렇다면 이제 마지막 페이지를 찾을 차례입니다.

#### 마지막 페이지 번호 찾기

![last_page](webcrawling/get_last_page.png)

In [34]:
# 역시 해당 tag의 xpath를 복사해옵니다. 하지만 규칙이 간단해서 xpath를 굳이 사용하진 않겠습니다.

# /html/body/div/table[2]/tbody/tr/td[7]/a

# td tag중에 class가 pgRR인 태그를 찾아서, 그 하위에 있는 a tag의 href 속성값을 가져옵니다.
last_url = source.find_all("td", class_="pgRR")[0].find_all("a")[0]["href"]
last_url

'/sise/sise_index_day.naver?code=KPI200&page=707'

- 위에 최종적으로 가져온 문자열에서 마지막 3글자를 때어내면 우리가 원하는 마지막 페이지 숫자인 591이 나옵니다.

In [38]:
# 마지막 글자를 때어내기 위해서 규칙을 생각해봅니다.
last_page = int(last_url.split('&page=')[-1])
last_page

707

- 진짜 마지막 단계입니다. 해당 페이지 주소의 규칙을 찾아야 합니다.


- 이제 크롤링의 과정은 다음과 같습니다.

1) 일별 시세 정보가 있는 페이지에 접속한다.


2) 페이지에서 날짜 / 체결가가 들어있는 태그를 검색한다.


3) 태그 중에서 우리가 찾은 조건에 맞는 데이터만 가져온다.


4) 이를 원하는만큼(현재는 마지막페이지까지) 반복한다.

In [ ]:
# 100 페이지에 접속하는 예시
naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(100)

# 위에서 작성했던 모든 코드를 종합합니다.



## 1.3 가져온 모든 정보를 하나로 합쳐서 구현하기 

In [39]:
# 위에서 구현한 모든 내용을 하나의 코드로 구현합니다.
date_list = []
price_list = []

for page_no in range(1, last_page+1):
    page_url = f"https://finance.naver.com/sise/sise_index_day.naver?code=KPI200&page={page_no}"

    source = requests.get(page_url).text
    source = bs4.BeautifulSoup(source)

    dates = source.find_all("td", class_="date")

    for date in dates:
        date_list.append(date.text)

    prices = source.find_all("td", class_="number_1")
    
    for price in prices[::4]:
        price_list.append(price.text)

In [41]:
len(date_list), len(price_list)

(4242, 4242)

In [43]:
date_list

['2023.02.22',
 '2023.02.21',
 '2023.02.20',
 '2023.02.17',
 '2023.02.16',
 '2023.02.15',
 '2023.02.14',
 '2023.02.13',
 '2023.02.10',
 '2023.02.09',
 '2023.02.08',
 '2023.02.07',
 '2023.02.06',
 '2023.02.03',
 '2023.02.02',
 '2023.02.01',
 '2023.01.31',
 '2023.01.30',
 '2023.01.27',
 '2023.01.26',
 '2023.01.25',
 '2023.01.20',
 '2023.01.19',
 '2023.01.18',
 '2023.01.17',
 '2023.01.16',
 '2023.01.13',
 '2023.01.12',
 '2023.01.11',
 '2023.01.10',
 '2023.01.09',
 '2023.01.06',
 '2023.01.05',
 '2023.01.04',
 '2023.01.03',
 '2023.01.02',
 '2022.12.29',
 '2022.12.28',
 '2022.12.27',
 '2022.12.26',
 '2022.12.23',
 '2022.12.22',
 '2022.12.21',
 '2022.12.20',
 '2022.12.19',
 '2022.12.16',
 '2022.12.15',
 '2022.12.14',
 '2022.12.13',
 '2022.12.12',
 '2022.12.09',
 '2022.12.08',
 '2022.12.07',
 '2022.12.06',
 '2022.12.05',
 '2022.12.02',
 '2022.12.01',
 '2022.11.30',
 '2022.11.29',
 '2022.11.28',
 '2022.11.25',
 '2022.11.24',
 '2022.11.23',
 '2022.11.22',
 '2022.11.21',
 '2022.11.18',
 '2022.11.

In [44]:
price_list

['315.94',
 '321.56',
 '321.79',
 '320.78',
 '324.10',
 '317.99',
 '323.47',
 '321.75',
 '323.69',
 '324.90',
 '325.63',
 '320.55',
 '318.83',
 '325.86',
 '323.93',
 '321.19',
 '317.26',
 '322.67',
 '327.98',
 '325.57',
 '320.76',
 '315.58',
 '313.81',
 '311.91',
 '313.75',
 '316.04',
 '313.77',
 '310.70',
 '310.38',
 '309.57',
 '309.69',
 '301.53',
 '297.87',
 '295.98',
 '289.58',
 '289.79',
 '291.10',
 '297.09',
 '304.22',
 '302.27',
 '302.07',
 '307.48',
 '303.23',
 '304.05',
 '306.77',
 '307.48',
 '307.73',
 '313.23',
 '309.56',
 '309.64',
 '311.63',
 '308.24',
 '308.27',
 '310.14',
 '313.91',
 '315.02',
 '322.02',
 '321.00',
 '315.58',
 '312.21',
 '316.07',
 '317.30',
 '314.11',
 '312.02',
 '314.67',
 '317.12',
 '316.75',
 '321.96',
 '322.31',
 '321.53',
 '323.09',
 '311.13',
 '315.23',
 '312.01',
 '307.87',
 '304.33',
 '301.66',
 '303.87',
 '303.88',
 '299.58',
 '295.06',
 '298.82',
 '293.85',
 '291.58',
 '291.47',
 '288.57',
 '288.61',
 '291.29',
 '293.59',
 '289.57',
 '289.36',

In [48]:
import pandas as pd

df = pd.DataFrame({"date": date_list, "price": price_list}).dropna()

df

,date,price
0,2023.02.22,315.94
1,2023.02.21,321.56
2,2023.02.20,321.79
3,2023.02.17,320.78
4,2023.02.16,324.10
...,...,...
4237,,
4238,,
4239,,
4240,,


In [49]:
df.to_excel("kpi200.xlsx", index=False)